In [3]:
import weka.core.jvm as jvm
import os
import traceback
import weka.core.jvm as jvm
from weka.core.converters import Loader
from weka.core.dataset import Instances
from weka.timeseries import TSEvaluation, TSEvalModule, WekaForecaster,Periodicity
from weka.classifiers import Classifier
from weka.core.classes import serialization_write, serialization_read
jvm.start(packages=True)

INFO:weka.core.jvm:JVM already running, call jvm.stop() first


In [4]:
# load a dataset
from weka.core.converters import Loader
loader = Loader(classname="weka.core.converters.ArffLoader")
airline_data = loader.load_file("data/TSLA-all.arff")
# airline_data.class_is_last()

In [5]:
# available evaluation modules
print("Evaluation modules")
modules = TSEvalModule.module_list()
print("Available modules")
for module in modules:
    print("-" + str(module))
print("Loading module by name")
print(TSEvalModule.module("MAE"))

Evaluation modules
Available modules
-Error
-MAE
-MSE
-RMSE
-MAPE
-DAC
-RAE
-RRSE
Loading module by name
MAE


In [6]:
# evaluate forecaster
print("Evaluate forecaster")
forecaster = WekaForecaster()
forecaster.fields_to_forecast = ["Adj Close"]
forecaster.base_forecaster = Classifier(classname="weka.classifiers.functions.LinearRegression")
forecaster.tslag_maker.timestamp_field = "Date"
forecaster.tslag_maker.adjust_for_variance = False

# add overlay data
forecaster.overlay_fields="Open,High,Low,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD"
print(forecaster.is_using_overlay_data)
print("forecast with overlays")
print(forecaster.forecast_with_overlays)

forecaster.tslag_maker.max_lag=7
forecaster.tslag_maker.min_lag=1
forecaster.tslag_maker.periodicity= Periodicity(periodicity="DAILY")



# print(help(forecaster.tslag_maker.periodicity))
# periodicity=None

forecaster.tslag_maker.include_powers_of_time = True
forecaster.tslag_maker.include_timelag_products = True
forecaster.tslag_maker.remove_leading_instances_with_unknown_lag_values = True

# forecaster.tslag_maker.add_month_of_year = True
# forecaster.tslag_maker.add_quarter_of_year = True

forecaster.tslag_maker.add_day_of_week=True
forecaster.tslag_maker.add_weekend_indicator=True

print("algorithm name: " + str(forecaster.algorithm_name))
print("command-line: " + forecaster.to_commandline())
print("lag maker: " + forecaster.tslag_maker.to_commandline())

evaluation = TSEvaluation(airline_data, 0.1)
evaluation.evaluate_on_training_data = True
evaluation.evaluate_on_test_data = True
# evaluation.prime_window_size = forecaster.tslag_maker.max_lag
evaluation.prime_for_test_data_with_test_data = True
evaluation.rebuild_model_after_each_test_forecast_step = False
evaluation.forecast_future = True
evaluation.horizon = 1
evaluation.prime_window_size=7
evaluation.evaluation_modules = "MAE,RMSE"


evaluation.evaluate(forecaster)
print(evaluation)
if evaluation.evaluate_on_training_data or evaluation.evaluate_on_test_data:
    print(evaluation.summary())
# if evaluation.evaluate_on_training_data:
#     print("Predictions (training data): " + evaluation.predictions_for_training_data(1).summary)
# if evaluation.evaluate_on_test_data:
#     print("Predictions (test data): " + evaluation.predictions_for_test_data(1).summary)
#     preds = evaluation.predictions_for_test_data(1)
#     print("Counts for targets: " + str(preds.counts_for_targets()))
#     print("Errors for target 'Adj Close': " + str(preds.errors_for_target("Adj Close")))
#     print("Errors for all targets: " + str(preds.predictions_for_all_targets()))
# if evaluation.training_data is not None:
#     print("Future forecasts (training)\n" + evaluation.print_future_forecast_on_training_data(forecaster))
# if evaluation.test_data is not None:
#     print("Future forecasts (test)\n" + evaluation.print_future_forecast_on_test_data(forecaster))

# if evaluation.evaluate_on_training_data:
#     print(evaluation.print_predictions_for_training_data("Predictions (training)", "Adj Close", 1))

# if evaluation.evaluate_on_test_data:
#     print(evaluation.print_predictions_for_test_data("Predictions (test)", "Adj Close", 1))

Evaluate forecaster
True
forecast with overlays
<bound method OverlayForecaster.forecast_with_overlays of Forecaster has not been built yet!>
algorithm name: LinearRegression -S 0 -R 1.0E-8 -num-decimal-places 4
command-line: weka.classifiers.timeseries.WekaForecaster -F "Adj Close" -O Open,High,Low,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD -trim-leading -L 1 -M 7 -B 2 -C 2 -G Date -dayofweek -weekend -conf 0 -P 0.95 -W "weka.classifiers.functions.LinearRegression -S 0 -R 1.0E-8 -num-decimal-places 4"
lag maker: weka.filters.supervised.attribute.TSLagMaker -F "Adj Close" -O Open,High,Low,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD -trim-leading -L 1 -M 7 -G Date -dayofweek -weekend
=== Evaluation setup ===

Relation: TSLA-all
# Training instances: 4234
# Test instances: 471
Evaluate on training data: True
Evaluate on test data: True
Horizon: 1
Prime window size: 7
Prime for test data with test data: True
Rebuild model after each tes

In [7]:
# build forecaster
print("Build/use forecaster")
airline_train, airline_test = airline_data.train_test_split(90.0)
forecaster = WekaForecaster()
forecaster.fields_to_forecast = ["Adj Close"]
forecaster.base_forecaster = Classifier(classname="weka.classifiers.functions.LinearRegression")
forecaster.fields_to_forecast = "Adj Close"
forecaster.build_forecaster(airline_train)
num_prime_instances = 10
airline_prime = Instances.copy_instances(airline_train, airline_train.num_instances - num_prime_instances, num_prime_instances)
forecaster.prime_forecaster(airline_prime)
num_future_forecasts = airline_test.num_instances
preds = forecaster.forecast(num_future_forecasts)
print("Actual,Predicted,Error")
for i in range(num_future_forecasts):
    actual = airline_test.get_instance(i).get_value(0)
    predicted = preds[i][0].predicted
    error = actual - predicted
    print("%f,%f,%f" % (actual, predicted, error))

Build/use forecaster
Actual,Predicted,Error
1646841600000.000000,0.632762,1646841599999.367188
1646928000000.000000,0.602561,1646927999999.397461
1647187200000.000000,0.597436,1647187199999.402588
1647273600000.000000,0.602570,1647273599999.397461
1647360000000.000000,0.605803,1647359999999.394287
1647446400000.000000,0.607183,1647446399999.392822
1647532800000.000000,0.611866,1647532799999.388184
1647792000000.000000,0.608290,1647791999999.391602
1647878400000.000000,0.610568,1647878399999.389404
1647964800000.000000,0.612885,1647964799999.387207
1648051200000.000000,0.620663,1648051199999.379395
1648137600000.000000,0.629036,1648137599999.370850
1648396800000.000000,0.631697,1648396799999.368408
1648483200000.000000,0.633094,1648483199999.366943
1648569600000.000000,0.633405,1648569599999.366699
1648656000000.000000,0.635465,1648655999999.364502
1648742400000.000000,0.637373,1648742399999.362549
1649001600000.000000,0.639525,1649001599999.360596
1649088000000.000000,0.641308,16490879

In [8]:
# serialization (if supported)
print("Serialization")
model_file = "base.model"
if forecaster.base_model_has_serializer:
    forecaster.save_base_model(model_file)
    forecaster2 = WekaForecaster()
    forecaster2.load_base_model(model_file)
    print(forecaster2.to_commandline())
else:
    print("Base model has no serializer, falling back to generic serialization")
    serialization_write(model_file, forecaster.base_forecaster)
    cls = Classifier(jobject=serialization_read(model_file))
    print(cls.to_commandline())

Serialization
Base model has no serializer, falling back to generic serialization
weka.classifiers.functions.LinearRegression -S 0 -R 1.0E-8 -num-decimal-places 4


In [9]:
# state management
print("State")
model_file = "state.ser"
if forecaster.uses_state:
    forecaster.serialize_state(model_file)
    forecaster2 = WekaForecaster()
    forecaster2.load_serialized_state(model_file)
    print(forecaster2.to_commandline())
else:
    print("Forecaster does not use state, falling back to generic serialization")
    serialization_write(model_file, forecaster)
    forecaster2 = WekaForecaster(jobject=serialization_read(model_file))
    print(forecaster2.to_commandline())

State
Forecaster does not use state, falling back to generic serialization
weka.classifiers.timeseries.WekaForecaster -F "Adj Close" -L 1 -M 12 -B 2 -C 2 -G ArtificialTimeIndex -conf 0 -P 0.95 -W "weka.classifiers.functions.LinearRegression -S 0 -R 1.0E-8 -num-decimal-places 4"


In [ ]:
jvm.stop()